In [44]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch
import torch.nn.functional as F
import csv

# Load pretrained model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)
model.eval()

def masked_tokens_probabilities(sentence: str, M, word_prob, i):
    """
    Calculate probabilities for multiple (mask_index, token) queries in one forward pass.

    Parameters:
    - sentence (str): Sentence containing one or more <mask> tokens.
    - queries (list of tuples): Each tuple = (mask_index, token).
        Example: [(0, "Paris"), (1, "Berlin")]

    Returns:
    - dict: { (mask_index, token): probability }
    """
    # Tokenize once
    inputs = tokenizer(sentence, return_tensors="pt")

    # Find all mask positions
    mask_positions = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=False)

    # Forward pass once
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)  # shape: [batch, seq_len, vocab_size]

  #  print(mask_positions[:, 1][-1])

    j = -1

    with open("test_NGSL_2.csv", newline="") as f:
        reader = csv.reader(f)
        for pos in mask_positions[:, 1]:
            if pos > 2 and pos < (mask_positions[:, 1][-1] - 1):
                for row in reader:
                    j+=1
                    for tok in row:
                        token_id = tokenizer.convert_tokens_to_ids(tok)
                        prob = probs[0, pos, token_id].item()
                        M[i][j] += word_prob * prob


            

        

"""
    results = {}
    for mask_index, token in queries:
        if mask_index >= len(mask_positions):
            raise ValueError(f"Mask index {mask_index} out of range. Found {len(mask_positions)} masks.")

        pos = mask_positions[mask_index, 1]  # token position of that mask
        token_id = tokenizer.convert_tokens_to_ids(token)
        prob = probs[0, pos, token_id].item()
        results[(mask_index, token)] = prob

    return results   """

'\n    results = {}\n    for mask_index, token in queries:\n        if mask_index >= len(mask_positions):\n            raise ValueError(f"Mask index {mask_index} out of range. Found {len(mask_positions)} masks.")\n\n        pos = mask_positions[mask_index, 1]  # token position of that mask\n        token_id = tokenizer.convert_tokens_to_ids(token)\n        prob = probs[0, pos, token_id].item()\n        results[(mask_index, token)] = prob\n\n    return results   '

In [45]:
from wordfreq import word_frequency

# Function to compute frequency of a word
def get_word_frequency(word: str, lang: str = "en") -> float:
    """
    Returns the frequency of a word in the given language.
    By default, English ("en") is used.
    
    The result is a probability between 0 and 1.
    """
    return word_frequency(word, lang)


In [52]:
import numpy as np
import csv

with open("test_NGSL.csv", newline="") as f:
    n = sum(1 for _ in f)

M = np.zeros((n, n), dtype=np.float64)

i=-1

with open("test_NGSL.csv", newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        i+=1
        for word in row:
            if(word != ""):
                word_prob = get_word_frequency(word, "en")
                masked_tokens_probabilities("<mask> <mask> <mask>" + word + "<mask> <mask> <mask> <mask> <mask> <mask>", M, word_prob, i)
            
            



In [53]:
M

array([[1.12963541e-13, 5.64817705e-14, 1.12963541e-13],
       [6.84060873e-13, 3.42030436e-13, 6.84060873e-13],
       [6.09340342e-13, 3.04670171e-13, 6.09340342e-13]])